In [1]:
from peft import (LoraConfig,IA3Config, PeftType, PrefixTuningConfig,
                  PromptEncoderConfig, PromptTuningConfig, TaskType,
                  get_peft_config, get_peft_model, get_peft_model_state_dict,
                  prepare_model_for_int8_training,
                  prepare_model_for_kbit_training, set_peft_model_state_dict)
from scipy.special import softmax
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (AutoModelForSequenceClassification,
                          AutoModelForTokenClassification, AutoTokenizer,
                          DataCollatorForTokenClassification,
                          LlamaForSequenceClassification, LlamaTokenizer,
                          Trainer, TrainingArguments,
                          get_linear_schedule_with_warmup, set_seed)
from model_utils import count_trainable_parameters, get_model_size, get_full_model_size
import pandas as pd
from tqdm import tqdm
import yaml
# import sys and append path
import sys
sys.path.append("../")
from peft_trainer import create_peft_config
from loguru import logger as loguru_logger
from argparse import Namespace

In [2]:

arg_dict = {
  "log_save_dir": "/mnt/sdc/niallt/saved_models/peft_training/logs",
  "task_type": "SEQ_CLS",
  "peft_method": "LORA",
  "lora_rank": 8, 
  "lora_alpha": 16,
  "lora_dropout": 0.1,
  "learning_rate": 3e-4,
  "num_virtual_tokens": 10 
}

# convert above args to a namespace object

args = Namespace(**arg_dict)

In [3]:
# model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-2-7b-hf")

In [4]:
# model

In [5]:
# create args


In [3]:


model_type_mappings = {"roberta": "roberta-base",
               "mobile": "nlpie/bio-mobilebert",
               "distil": "nlpie/distil-biobert",
               "tiny": "nlpie/tiny-biobert",
               "llama-7b": "meta-llama/Llama-2-7b-hf",
               "bert": "dmis-lab/biobert-v1.1",
               }

peft_types = ["PROMPT_TUNING","LORA", "PREFIX_TUNING", "P_TUNING"]

def get_number_of_trainable_params(model_type_mappings:dict,
                                   peft_types:list,
                                   task_type:str = "SEQ_CLS",
                                   num_labels:int = 2):

    # set up empty dicts to full for dfs
    model_peft_dict = {}
    
    for model_type in model_type_mappings.keys():
        
        model_dict = {}
        model_name_or_path = model_type_mappings[model_type]
        model_args = dict(pretrained_model_name_or_path=model_name_or_path, 
                          num_labels=num_labels, 
                          output_hidden_states=False, 
                          trust_remote_code=True)

            
        if task_type == "SEQ_CLS":
            model = AutoModelForSequenceClassification.from_pretrained(**model_args)
        elif task_type == "TOKEN_CLS":
            model = AutoModelForTokenClassification.from_pretrained(**model_args)
        
        # falcon model seems to use model config to define pad token and the remote code panicks if you don't set it
        if "falcon" in model_name_or_path:
            model.config.use_cache = False            

        # count total trainable params before peft
        total_trainable_params = count_trainable_parameters(model)
        
        # get model size and full model size too
        model_size_MB, model_size_GB = get_model_size(model)
        full_model_size_MB, full_model_size_GB = get_full_model_size(model)
        
        for peft_method in tqdm(peft_types, desc=f"model type: {model_type}"):
            
            
            # set up some PEFT params
            peft_config, lr = create_peft_config(args, peft_method, model_name_or_path,task_type)
            model = get_peft_model(model, peft_config)
            print(f"peft config is: {peft_config}")
            # print(model)
            model.print_trainable_parameters()
            
            # lets also confirm this directly and save to args
            n_trainable_params = count_trainable_parameters(model)
            # proportion of total trainable params
            n_trainable_params_perc = (n_trainable_params / total_trainable_params) * 100
            
            # get size of peft adapter only
            peft_model_size_MB, peft_model_size_GB = get_model_size(model)
            peft_full_model_size_MB, peft_full_model_size_GB = get_full_model_size(model)
            
            # store the model name, peft method and number of trainable params
            model_dict[peft_method] = {"n_trainable_params": n_trainable_params,
                                 "total_trainable_params": total_trainable_params,
                                 "n_trainable_params_perc": n_trainable_params_perc,
                                 "model_size_MB": model_size_MB,
                                 "model_size_GB": model_size_GB,
                                 "full_model_size_MB": full_model_size_MB,
                                 "full_model_size_GB": full_model_size_GB,
                                 "peft_model_size_MB": peft_model_size_MB,
                                 "peft_model_size_GB": peft_model_size_GB,
                                 "peft_full_model_size_MB": peft_full_model_size_MB,
                                 "peft_full_model_size_GB": peft_full_model_size_GB,}
            
        model_peft_dict[model_type] = model_dict

    return model_peft_dict

In [4]:
all_df = get_number_of_trainable_params(model_type_mappings, peft_types)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model size (MB): 475.49121856689453
Total size (MB): 951.0656957626343


model type: roberta:   0%|          | 0/4 [00:00<?, ?it/s]2023-11-10 11:27:50.842 | INFO     | peft_trainer:create_peft_config:705 - Using PROMPT_TUNING


peft config is: PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path='roberta-base', revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=768, num_transformer_submodules=1, num_attention_heads=12, num_layers=12, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None)
trainable params: 599,810 || all params: 125,246,980 || trainable%: 0.4789017667332178
Model size (MB): 2.2880935668945312


model type: roberta:  25%|██▌       | 1/4 [00:00<00:01,  1.98it/s]2023-11-10 11:27:51.349 | INFO     | peft_trainer:create_peft_config:648 - Using LORA


...

Total size (MB): 415.59956455230713
peft config is: PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=768, num_transformer_submodules=1, num_attention_heads=12, num_layers=12, encoder_hidden_size=768, prefix_projection=False)
trainable params: 185,858 || all params: 108,792,580 || trainable%: 0.1708370184804883
Model size (MB): 0.7089920043945312


model type: bert:  75%|███████▌  | 3/4 [00:01<00:00,  1.92it/s]2023-11-10 11:31:00.482 | INFO     | peft_trainer:create_peft_config:719 - Using P_TUNING


Total size (MB): 415.85884189605713
peft config is: PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, num_virtual_tokens=10, token_dim=768, num_transformer_submodules=1, num_attention_heads=12, num_layers=12, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=128, encoder_num_layers=2, encoder_dropout=0.0)
trainable params: 223,234 || all params: 108,829,956 || trainable%: 0.20512183244841153
Model size (MB): 0.8515701293945312


model type: bert: 100%|██████████| 4/4 [00:02<00:00,  1.92it/s]

Total size (MB): 416.15399646759033


In [12]:
all_df

{'vanilla': {'PROMPT_TUNING': {'n_trainable_params': 1191940,
   'total_trainable_params': 124647170,
   'n_trainable_params_perc': 0.956251152753809,
   'model_size_MB': 475.49121856689453,
   'model_size_GB': 0.46434689313173294,
   'full_model_size_MB': 951.0699663162231,
   'full_model_size_GB': 0.9287792639806867,
   'peft_model_size_MB': 4.5468902587890625,
   'peft_model_size_GB': 0.004440322518348694,
   'peft_full_model_size_MB': 482.42061138153076,
   'peft_full_model_size_GB': 0.47111387830227613},
  'LORA': {'n_trainable_params': 294912,
   'total_trainable_params': 124647170,
   'n_trainable_params_perc': 0.23659742936803138,
   'model_size_MB': 475.49121856689453,
   'model_size_GB': 0.46434689313173294,
   'full_model_size_MB': 951.0699663162231,
   'full_model_size_GB': 0.9287792639806867,
   'peft_model_size_MB': 1.125,
   'peft_model_size_GB': 0.0010986328125,
   'peft_full_model_size_MB': 480.1683073043823,
   'peft_full_model_size_GB': 0.46891436260193586},
  'PREFI

In [5]:
# write to file
with open("../model_type_trainable_model_size.yaml", "w") as f:
    yaml.dump(all_df, f)
    


In [4]:
# reload that yaml file
with open("../model_type_trainable_model_size.yaml", "r") as f:
    all_df = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
# run through they dict keys and nicely print the attributes and values
for model_type in all_df.keys():
    print(f"###### model type: {model_type} ######### \n")
    for peft_method in all_df[model_type].keys():
        print(f"peft method: {peft_method}")
        for attribute in all_df[model_type][peft_method].keys():
            print(f"{attribute}: {all_df[model_type][peft_method][attribute]}")
        print("\n")

###### model type: bert ######### 

peft method: LORA
full_model_size_GB: 0.807065955363214
full_model_size_MB: 826.4355382919312
model_size_GB: 0.40349293500185013
model_size_MB: 413.17676544189453
n_trainable_params: 296450
n_trainable_params_perc: 0.2737005318256615
peft_full_model_size_GB: 0.40585894975811243
peft_full_model_size_MB: 415.59956455230713
peft_model_size_GB: 0.0011043623089790344
peft_model_size_MB: 1.1308670043945312
total_trainable_params: 108311810


peft method: PREFIX_TUNING
full_model_size_GB: 0.807065955363214
full_model_size_MB: 826.4355382919312
model_size_GB: 0.40349293500185013
model_size_MB: 413.17676544189453
n_trainable_params: 185858
n_trainable_params_perc: 0.17159532279997905
peft_full_model_size_GB: 0.4061121502891183
peft_full_model_size_MB: 415.85884189605713
peft_model_size_GB: 0.0006923750042915344
peft_model_size_MB: 0.7089920043945312
total_trainable_params: 108311810


peft method: PROMPT_TUNING
full_model_size_GB: 0.807065955363214
full_model